In [1]:
# libraries

import osmnx as ox
import geopandas as gpd
import folium
import pandas as pd

In [2]:
#extract Tuaranga road geometries

G = ox.graph_from_place("Tauranga City, New Zealand", network_type='drive')

In [3]:
#import data

hybrid_services = gpd.read_file(r"C:\Users\GSE1\Beca\3813283 - PT Services and Infrastructure SSBC - Documents\Technical - Working Files\01 SSBC\05 Shortlist\01 Service options assessment\hybrid_services.geojson")
pt_demand = gpd.read_file(r'C:\Users\GSE1\Beca\3813283 - PT Services and Infrastructure SSBC - Documents\Technical - Working Files\01 SSBC\05 Shortlist\01 Service options assessment\pt_trip_2048.csv')
los_links_am = gpd.read_file(r'\\Beca.net\projects\381\3813283\4-TTR\Modelling\Outputs\Shapefiles_18Aug2023\LOS_Toll_AM_Links_2048DN_TTM22_NoSB.shp', geometry="geometry")
los_nodes_am = gpd.read_file(r'\\Beca.net\projects\381\3813283\4-TTR\Modelling\Outputs\Shapefiles_18Aug2023\LOS_Toll_AM_Nodes_2048DN_TTM22_NoSB.shp', geometry="geometry")

In [4]:
#set crs
los_links_am.set_crs('epsg:2193', inplace=True)
los_nodes_am.set_crs('epsg:2193', inplace=True)
pt_demand.set_crs('epsg:2193', inplace=True)
hybrid_services.to_crs(crs=2193, inplace=True)

In [5]:
#create geodataframe of all road geometries in Tauranga

gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)

In [6]:
gdf.head()

osmid   lanes   
u         v          key                                               
247014753 7397434438 0                             657538168       2  \
          4866204635 0    [701547512, 461219596, 1107883893]  [1, 2]   
247014899 247014903  0                [1103106292, 27212758]  [1, 3]   
          1597355601 0                             146443707       2   
247014903 298632431  0                             392378544     NaN   

                                       name    highway maxspeed  oneway   
u         v          key                                                  
247014753 7397434438 0      Te Puke Highway  secondary      100   False  \
          4866204635 0                  NaN  secondary       70    True   
247014899 247014903  0    State Highway 29A      trunk       50    True   
          1597355601 0    State Highway 29A      trunk      100   False   
247014903 298632431  0                  NaN      trunk       50    True   

                         reversed    length   
u         v          key                      
247014753 7397434438 0       True   585.773  \
          4866204635 0      False   123.800   
247014899 247014903  0      False   349.942   
          1597355601 0       True  1351.773   
247014903 298632431  0      False    12.105   

                                                                   geometry   
u         v          key                                                      
247014753 7397434438 0    LINESTRING (176.28407 -37.71666, 176.28416 -37...  \
          4866204635 0    LINESTRING (176.28407 -37.71666, 176.28393 -37...   
247014899 247014903  0    LINESTRING (176.17417 -37.72352, 176.17384 -37...   
          1597355601 0    LINESTRING (176.17417 -37.72352, 176.17432 -37...   
247014903 298632431  0    LINESTRING (176.17059 -37.72473, 176.17046 -37...   

                             ref bridge  junction width area access  
u         v          key                                             
247014753 7397434438 0       NaN    NaN       NaN   NaN  NaN    NaN  
          4866204635 0       NaN    NaN       NaN   NaN  NaN    NaN  
247014899 247014903  0    SH 29A    NaN       NaN   NaN  NaN    NaN  
          1597355601 0    SH 29A    NaN       NaN   NaN  NaN    NaN  
247014903 298632431  0    SH 29A    yes  circular   NaN  NaN    NaN

In [7]:
#drop na values from gdf column'lanes', set CRS

gdf_drop = gdf.dropna(subset=['lanes'])
gdf_drop.head()
gdf_drop.set_crs('epsg:2193', inplace=True, allow_override=True)

C:\Users\GSE1\.conda\envs\ox\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


osmid   lanes   
u           v           key                                               
247014753   7397434438  0                             657538168       2  \
            4866204635  0    [701547512, 461219596, 1107883893]  [1, 2]   
247014899   247014903   0                [1103106292, 27212758]  [1, 3]   
            1597355601  0                             146443707       2   
247014956   3777941155  0                             374405015       3   
...                                                         ...     ...   
10235183745 9499693321  0                             755863781       1   
10235183746 10235183752 0                             755863780       1   
10235183752 7059762011  0                             755863780       1   
10235183753 10235183745 0                             755863781       1   
10235226142 10145438695 0                            1119121977       2   

                                          name    highway maxspeed  oneway   
u           v           key                                                  
247014753   7397434438  0      Te Puke Highway  secondary      100   False  \
            4866204635  0                  NaN  secondary       70    True   
247014899   247014903   0    State Highway 29A      trunk       50    True   
            1597355601  0    State Highway 29A      trunk      100   False   
247014956   3777941155  0    State Highway 29A      trunk       50    True   
...                                        ...        ...      ...     ...   
10235183745 9499693321  0      Te Okuroa Drive  secondary       50    True   
10235183746 10235183752 0      Te Okuroa Drive  secondary       50    True   
10235183752 7059762011  0      Te Okuroa Drive  secondary       50    True   
10235183753 10235183745 0      Te Okuroa Drive  secondary       50    True   
10235226142 10145438695 0      Taurikura Drive  secondary      NaN    True   

                            reversed    length   
u           v           key                      
247014753   7397434438  0       True   585.773  \
            4866204635  0      False   123.800   
247014899   247014903   0      False   349.942   
            1597355601  0       True  1351.773   
247014956   3777941155  0      False    38.873   
...                              ...       ...   
10235183745 9499693321  0      False   171.369   
10235183746 10235183752 0      False     8.612   
10235183752 7059762011  0      False    24.754   
10235183753 10235183745 0      False     8.458   
10235226142 10145438695 0      False    26.442   

                                                                      geometry   
u           v           key                                                      
247014753   7397434438  0    LINESTRING (176.284 -37.717, 176.284 -37.717, ...  \
            4866204635  0    LINESTRING (176.284 -37.717, 176.284 -37.717, ...   
247014899   247014903   0    LINESTRING (176.174 -37.724, 176.174 -37.724, ...   
            1597355601  0    LINESTRING (176.174 -37.724, 176.174 -37.723, ...   
247014956   3777941155  0    LINESTRING (176.168 -37.727, 176.168 -37.727, ...   
...                                                                        ...   
10235183745 9499693321  0    LINESTRING (176.348 -37.731, 176.347 -37.731, ...   
10235183746 10235183752 0        LINESTRING (176.348 -37.731, 176.348 -37.731)   
10235183752 7059762011  0        LINESTRING (176.348 -37.731, 176.348 -37.731)   
10235183753 10235183745 0        LINESTRING (176.348 -37.731, 176.348 -37.731)   
10235226142 10145438695 0    LINESTRING (176.087 -37.756, 176.087 -37.756, ...   

                                ref bridge junction width area access  
u           v           key                                            
247014753   7397434438  0       NaN    NaN      NaN   NaN  NaN    NaN  
            4866204635  0       NaN    NaN      NaN   NaN  NaN    NaN  
247014899   247014903   0    SH 29A    NaN      NaN   NaN  Na

In [8]:
# filter by lanes and include 'oneway' only

two_lane = gdf_drop.loc[(gdf_drop['lanes']=='2')&(gdf_drop['oneway']=='True')]
three_lane = gdf_drop.loc[(gdf_drop['lanes']=='3')&(gdf_drop['oneway']=='True')] 
four_lane = gdf_drop.loc[(gdf_drop['lanes']=='4')&(gdf_drop['oneway']=='True')] 
five_lane = gdf_drop.loc[(gdf_drop['lanes']=='5')&(gdf_drop['oneway']=='True')] 

In [9]:
# filter junctions

junction_circular = gdf_drop.loc[(gdf_drop['junction']=='circular')]
junction_roundabout = gdf_drop.loc[(gdf_drop['junction']=='roundabout')]

In [10]:
# filter by segment length: bus lane=0-25m,BRT=500m+, Busway=500m+ 

bus_lane = gdf_drop.loc[(gdf_drop['lanes']=='2')|(gdf_drop['lanes']=="['3','2']")&(gdf_drop['oneway']==True)]
brt = gdf_drop.loc[(gdf_drop['lanes']=='2')&(gdf_drop['oneway']==True)&(gdf_drop['length']>=500)]
busway = gdf_drop.loc[(gdf_drop['lanes']=='3')&(gdf_drop['oneway']==True)&(gdf_drop['length']>=500)]

In [11]:
#pt demand - drop all columns except PT and geometry. Set CRS

pt_demand = pt_demand.loc[:, ['ADT_PUBLIC', 'ADT_PT', 'AM_PT', 'IP_PT', 'PM_PT', 'geometry']]

In [12]:
pt_demand.columns

Index(['ADT_PUBLIC', 'ADT_PT', 'AM_PT', 'IP_PT', 'PM_PT', 'geometry'], dtype='object')

In [13]:
# change data types of columns to float

pt_demand[['ADT_PUBLIC', 'ADT_PT', 'AM_PT', 'IP_PT', 'PM_PT']] = pt_demand[['ADT_PUBLIC', 'ADT_PT', 'AM_PT', 'IP_PT', 'PM_PT']].astype(float)

In [14]:
# Create equal-sized bins of AM PT demand and create new column
pt_demand['PT_BIN'] = pd.cut(pt_demand['AM_PT'], bins=5, labels=False)

In [15]:
pt_demand['AM_PT'].describe()

count    5610.000000
mean       21.936649
std        51.931386
min         0.000000
25%         0.000000
50%         0.000000
75%        15.727500
max       432.620000
Name: AM_PT, dtype: float64

In [16]:
hybrid_services['vehicle_co'].describe()

count    13.000000
mean     12.230769
std       8.237656
min       3.000000
25%       5.000000
50%      11.000000
75%      20.000000
max      27.000000
Name: vehicle_co, dtype: float64

In [17]:
# only show links with demand higher than 75th percentile

high_pt = pt_demand.loc[(pt_demand['AM_PT']>16)]

In [18]:
high_vehicle = hybrid_services.loc[(hybrid_services['vehicle_co']>=20)]

In [19]:
high_vehicle.to_crs(2193, inplace=False)

,full_name,dir_id,dir_name,pattern,line_id,line_name,line_color,map_id,map_name,url,km,hours_year,cost_year,vehicle_ty,vehicle_co,minheadway,maxheadway,minkph,maxkph,geometry
6,C Pyes Pa - City - Papamoa - inbound,1190326835,inbound,A,8bb30649,C Pyes Pa - City - Papamoa,#A72051,18b88c2d,Hybrid Network 2048 1.1,https://platform.remix.com/project/10ce5503/li...,27.531,92177.616667,9.217762e+06,Bus,21,7,7,24.100000,24.100000,"MULTILINESTRING ((1889601.365 5822228.975, 188..."
8,BX Te Tumu - City - Bethlehem - inbound,1190326831,inbound,A,91d95ca8,BX Te Tumu - City - Bethlehem,#C9500E,18b88c2d,Hybrid Network 2048 1.1,https://platform.remix.com/project/10ce5503/li...,39.880,120332.666667,1.203327e+07,Bus,27,7,7,26.423667,26.423667,"MULTILINESTRING ((1900592.575 5816564.510, 190..."
11,F Beachfront - inbound,1190326833,inbound,A,c56c2241,F Beachfront,#5A8400,18b88c2d,Hybrid Network 2048 1.1,https://platform.remix.com/project/10ce5503/li...,27.901,92509.450000,9.250945e+06,Bus,21,7,7,24.100000,24.100000,"MULTILINESTRING ((1900588.399 5816567.089, 190..."
12,D Welcome Bay - City - Bethlehem - inbound,1190326841,inbound,A,cad900d2,D Welcome Bay - City - Bethlehem,#613FA6,18b88c2d,Hybrid Network 2048 1.1,https://platform.remix.com/project/10ce5503/li...,27.534,90166.541667,9.016654e+06,Bus,20,7,7,24.100000,24.100000,"MULTILINESTRING ((1882069.393 5817876.169, 188..."


In [20]:
high_vehicle.crs

<Projected CRS: EPSG:2193>
Name: NZGD2000 / New Zealand Transverse Mercator 2000
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: New Zealand - North Island, South Island, Stewart Island - onshore.
- bounds: (166.37, -47.33, 178.63, -34.1)
Coordinate Operation:
- name: New Zealand Transverse Mercator 2000
- method: Transverse Mercator
Datum: New Zealand Geodetic Datum 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [21]:
high_pt_bin = pt_demand.loc[(pt_demand['PT_BIN']>=2)]

In [22]:
los_nodes_am.columns

Index(['N', 'DELAY_MAX', 'LOS_MAX', 'DELAY_WAVG', 'LOS_WAVG', 'SECTOR', 'X',
       'Y', 'LOCATION', 'LABEL', 'FAREZONE', 'geometry'],
      dtype='object')

In [23]:
low_los_nodes = los_nodes_am.loc[(los_nodes_am['LOS_WAVG']>=3)]
low_los_links = los_links_am.loc[(los_links_am['LOS']>=3)]

In [24]:
# Folium map

m = high_pt_bin.explore(column="AM_PT",
    tiles="CartoDB positron",
    tooltip="AM_PT",
    highlight=True,
    name='AM peak PT Demand',
    k=5,
    popup=['AM_PT'])

buses = high_vehicle.explore(
    m=m,
    column="vehicle_co",
    tooltip="line_name",
    name="Hybrid Network Bus Routes",
    highlight=True)


los_nodes = low_los_nodes.explore(
    m=m,
    column="LOS_WAVG",
    tooltip="LOS_WAVG",
    name="LOS Nodes (AM)",
    highlight=True,
    marker_type="circle_marker",
    marker_kwds={
        "radius" : "6"})

los_links = low_los_links.explore(
    m=m,
    column="LOS",
    tooltip="LOS",
    name="LOS Links (AM)",
    highlight=True,)

folium.LayerControl().add_to(m)

m

In [25]:
pt_priority_intersection = high_pt_bin.overlay(low_los_nodes, how='intersection', keep_geom_type=False)

In [26]:
pt_priority_intersection.head()

,ADT_PUBLIC,ADT_PT,AM_PT,IP_PT,PM_PT,PT_BIN,N,DELAY_MAX,LOS_MAX,DELAY_WAVG,LOS_WAVG,SECTOR,X,Y,LOCATION,LABEL,FAREZONE,geometry
0,2424.95557,2424.95557,198.30,187.54,250.73,2,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000)
1,2390.43384,2451.89380,373.63,192.35,120.97,4,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000)
2,2369.28027,2369.28027,360.49,183.51,122.99,4,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000)
3,2461.48462,2490.00464,211.54,189.26,260.29,2,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000)
4,244.75258,472.17258,197.67,8.22,6.60,2,6311,30.6,3,25.37175,3,0,1872695.0,5823051.0,0,Minden,2,POINT (1872695.000 5823051.000)


In [27]:
pt_priority_intersection.columns

Index(['ADT_PUBLIC', 'ADT_PT', 'AM_PT', 'IP_PT', 'PM_PT', 'PT_BIN', 'N',
       'DELAY_MAX', 'LOS_MAX', 'DELAY_WAVG', 'LOS_WAVG', 'SECTOR', 'X', 'Y',
       'LOCATION', 'LABEL', 'FAREZONE', 'geometry'],
      dtype='object')

In [28]:
# rank nodes based on demand

pt_priority_intersection['rank'] = pt_priority_intersection['DELAY_WAVG'].rank(method="average", ascending=False).astype('int')

In [29]:
pt_priority_intersection.head()

,ADT_PUBLIC,ADT_PT,AM_PT,IP_PT,PM_PT,PT_BIN,N,DELAY_MAX,LOS_MAX,DELAY_WAVG,LOS_WAVG,SECTOR,X,Y,LOCATION,LABEL,FAREZONE,geometry,rank
0,2424.95557,2424.95557,198.30,187.54,250.73,2,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000),44
1,2390.43384,2451.89380,373.63,192.35,120.97,4,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000),44
2,2369.28027,2369.28027,360.49,183.51,122.99,4,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000),44
3,2461.48462,2490.00464,211.54,189.26,260.29,2,1431,49.2,4,32.36258,3,0,1879125.0,5824210.0,0,Cam/Elizabeth,2,POINT (1879125.000 5824210.000),44
4,244.75258,472.17258,197.67,8.22,6.60,2,6311,30.6,3,25.37175,3,0,1872695.0,5823051.0,0,Minden,2,POINT (1872695.000 5823051.000),60


In [30]:
#identify links where there is high PT demand and high number of buses

links_merged = high_pt_bin.sjoin(high_vehicle, how="left")

In [31]:
pt_priority_links = low_los_links.overlay(links_merged, how='intersection', keep_geom_type=True)

In [32]:
pt_priority_links.columns

Index(['A', 'B', 'FREIGHTLANE', 'BUSLANE', 'BL_IP', 'GRADIENT', 'FSFACTOR',
       'DISTANCE', 'LINKTYPE', 'JURISDICT', 'CAPINDEX', 'TSFLAG', 'HOVLANE',
       'ORG_CAP', 'MOD_CAP', 'SCHEME', 'DEL', 'LANEOP', 'ORG_FSPD', 'MOD_FSPD',
       'AKCEJ', 'LANENUM', 'TOLLENTRY', 'TOLLEXIT', 'TOLLROAD', 'TOLL',
       'HCVTOLL', 'IPFACTOR', 'LINKTAG', 'CAPACITY', 'FSPEED', 'CALDIS',
       'DIFFDIS', 'V_1', 'TIME_1', 'VC_1', 'CSPD_1', 'VDT_1', 'VHT_1', 'V1_1',
       'V2_1', 'V3_1', 'V4_1', 'V5_1', 'V6_1', 'V7_1', 'V8_1', 'V9_1', 'V10_1',
       'V11_1', 'V12_1', 'V13_1', 'V14_1', 'VT_1', 'V1T_1', 'V2T_1', 'V3T_1',
       'V4T_1', 'V5T_1', 'V6T_1', 'V7T_1', 'V8T_1', 'V9T_1', 'V10T_1',
       'V11T_1', 'V12T_1', 'V13T_1', 'V14T_1', 'LW_TTIME_1', 'TYPE', 'LOS',
       'ADT_PUBLIC', 'ADT_PT', 'AM_PT', 'IP_PT', 'PM_PT', 'PT_BIN',
       'index_right', 'full_name', 'dir_id', 'dir_name', 'pattern', 'line_id',
       'line_name', 'line_color', 'map_id', 'map_name', 'url', 'km',
       'hours_year', '

In [33]:
pt_priority_links['rank'] = pt_priority_links['LOS'].rank(method="average", ascending=False).astype('int')

In [34]:
pt_priority_intersection.shape

(78, 19)

In [35]:
pt_priority_links.shape

(179, 99)

# What happens when you prioritise locations by delay?

In [36]:
map = pt_priority_intersection.explore(tiles="CartoDB positron",
    column="rank",
    tooltip=['AM_PT', 'LOS_WAVG', "rank"],
    highlight=True,
    name="Nodes",
    marker_type="circle_marker",
    marker_kwds={
        "radius" : "6"},
    legend_kwds= {
        "caption" : "Nodes by priority rank"
    },
    cmap='viridis')

links = pt_priority_links.explore(
    m=map,
    column="rank",
    tooltip=["AM_PT", "LOS", "rank"],
    highlight=True,
    name="Links",
    legend_kwds= {
        "caption" : "Links by priority rank"
    })

folium.LayerControl().add_to(map)

map